<a href="https://colab.research.google.com/github/julle0123/CalMate-20B-KO-LoRA/blob/main/CalMate_20B_KO_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# 원하면 맨 앞에 %%capture 붙여서 깔끔하게 설치 로그 숨겨도 됩니다.
!pip install -q "numpy==2.0.2"
!pip install -q "torch==2.5.1" "torchvision==0.20.1" -i https://download.pytorch.org/whl/cu121

# Finetune에 필요한 핵심 스택 (Unsloth 호환 조합)
!pip install -q \
  "bitsandbytes==0.43.3" \
  "transformers==4.44.2" "tokenizers==0.20.1" \
  "trl==0.9.6" "peft==0.11.1" "accelerate==0.32.1" \
  "datasets==2.20.0" "evaluate==0.4.3" "jsonschema==4.22.0"

# Unsloth는 PyPI 안정 릴리스만 사용 (git 말고)
!pip install -q "unsloth" "unsloth_zoo"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 114.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 117.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch, transformers, tokenizers, bitsandbytes as bnb, numpy as np
print("GPU:", torch.cuda.get_device_name(0), "| CUDA:", torch.cuda.is_available())
print("torch:", torch.__version__, "| transformers:", transformers.__version__, "| tokenizers:", tokenizers.__version__)
print("numpy:", np.__version__, "| bitsandbytes:", bnb.__version__)


GPU: NVIDIA A100-SXM4-40GB | CUDA: True
torch: 2.7.1+cu126 | transformers: 4.55.0 | tokenizers: 0.21.4
numpy: 2.0.2 | bitsandbytes: 0.47.0


In [5]:
# ❗️반드시 transformers보다 먼저 가져오기
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import unsloth
from unsloth import FastLanguageModel

import torch
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer

# 안정 플래그(A100 권장)
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

max_seq_length = 1024
dtype = None
model_id = "unsloth/gpt-oss-20b-unsloth-bnb-4bit"   # 20B 4bit 변환본

# 4bit 로드
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    dtype = dtype,                  # None = auto
    max_seq_length = max_seq_length,
    load_in_4bit = True,            # 4bit 양자화
    full_finetuning = False,        # QLoRA
    # token = "hf_...",             # 필요 시 HF 토큰
)

# LoRA 적용
model = FastLanguageModel.get_peft_model(
    model,
    r = 8,                          # A100 여유→16도 OK, 시작은 8~16
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout = 0.0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",   # 긴 컨텍스트 최적화
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

# pad 토큰 보정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token


/tmp/ipython-input-3325015057.py:5: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.4: Fast Gpt_Oss patching. Transformers: 4.55.0.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.00G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth: Making `model.base_model.model.model` require gradients


In [7]:
# /content/data/calmate_train.jsonl 생성: 일정 비서 도메인 전용 합성 데이터(예시 300개)
import os, json, random
os.makedirs("/content/data", exist_ok=True)

def row(user, assistant):
    return {"messages":[
        {"role":"system","content":"너는 한국어 일정 비서다. 사용자의 요청을 이해해 일정 등록/수정/확인 답변을 간결하게 제공한다."},
        {"role":"user","content":user},
        {"role":"assistant","content":assistant},
    ]}

samples = [
    row("내일 오후 3시에 강남에서 팀미팅 2시간 잡아줘",
        "확인했어. '팀미팅'을 내일 15:00~17:00, 장소: 강남으로 등록해둘게. 변경사항 있으면 알려줘."),
    row("금요일 오전 치과 예약 1시간, 중요도 높음",
        "금요일 09:00~10:00 치과 진료로 등록하고 중요도를 높음으로 표시할게."),
]

days  = ["오늘","내일","모레","이번주 금요일","다음주 월요일","다음주 수요일"]
areas = ["강남","판교","역삼","잠실","본사","을지로"]
kinds = ["미팅","회의","치과 진료","면접","데모"]
for _ in range(298):
    d=random.choice(days); a=random.choice(areas); k=random.choice(kinds)
    hr=random.choice([9,10,11,13,14,15,16,19]); dur=random.choice([30,60,90,120])
    pri=random.choice(["", " 중요도 높음", " 중요도 낮음", " 우선순위 높음"]).strip()
    user=f"{d} {hr}시에 {a}에서 {k} {dur if dur<60 else dur//60}{'분' if dur<60 else '시간'}{(' ' + pri) if pri else ''}".strip()
    end_h, end_m = hr + dur//60, (dur%60)
    st_txt, end_txt = f"{hr:02d}:00", f"{end_h:02d}:{(end_m or 0):02d}"
    assistant=f"확인했어. '{k}'을 {d} {st_txt}~{end_txt}, 장소: {a}로 등록해둘게. 필요하면 수정해줘."
    samples.append(row(user, assistant))

with open("/content/data/calmate_train.jsonl","w",encoding="utf-8") as f:
    for s in samples:
        f.write(json.dumps(s, ensure_ascii=False)+"\n")

print("샘플 수:", len(samples), "| 파일:", "/content/data/calmate_train.jsonl")


샘플 수: 300 | 파일: /content/data/calmate_train.jsonl


In [8]:
from datasets import load_dataset

ds = load_dataset("json", data_files="/content/data/calmate_train.jsonl", split="train")
assert "messages" in ds.column_names, f"messages 컬럼이 필요합니다. 현재 컬럼: {ds.column_names}"

# tok는 이전에 로드한 tokenizer를 재사용해도 되고, 없으면 아래 두 줄로 로드
# from transformers import AutoTokenizer
# tok = AutoTokenizer.from_pretrained("unsloth/gpt-oss-20b-unsloth-bnb-4bit", use_fast=True, trust_remote_code=True)

def to_text(batch):
    texts=[]
    for convo in batch["messages"]:
        try:
            txt = tok.apply_chat_template(convo, tokenize=False, add_generation_prompt=False)
        except Exception:
            # 모델 템플릿이 없을 때 수동 하모니 포맷
            sys = next((m["content"] for m in convo if m["role"]=="system"), "")
            usr = next((m["content"] for m in convo if m["role"]=="user"), "")
            ass = next((m["content"] for m in convo if m["role"]=="assistant"), "")
            txt = f"<|system|>\n{sys}\n<|user|>\n{usr}\n<|assistant|>\n{ass}"
        texts.append(txt)
    return {"text": texts}

ds_text = ds.map(to_text, batched=True)  # remove_columns는 생략(검증용 미리보기 위해)
print(ds_text[0]["text"][:200])


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

<|system|>
너는 한국어 일정 비서다. 사용자의 요청을 이해해 일정 등록/수정/확인 답변을 간결하게 제공한다.
<|user|>
내일 오후 3시에 강남에서 팀미팅 2시간 잡아줘
<|assistant|>
확인했어. '팀미팅'을 내일 15:00~17:00, 장소: 강남으로 등록해둘게. 변경사항 있으면 알려줘.


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from trl import SFTTrainer, SFTConfig

model_id = "unsloth/gpt-oss-20b-unsloth-bnb-4bit"

# --- Tokenizer ---
try:
    tok
except NameError:
    try:
        tok = AutoTokenizer.from_pretrained(model_id, use_fast=True, trust_remote_code=True)
    except Exception:
        tok = AutoTokenizer.from_pretrained(model_id, use_fast=False, trust_remote_code=True)
    if tok.pad_token is None: tok.pad_token = tok.eos_token

# --- 4bit 로드 (fp16로 맞춤) ---
bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,   # ← fp16로 통일
)

try:
    base
except NameError:
    base = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_cfg,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        # 필요시: attn_implementation="eager",
    )
    base.gradient_checkpointing_enable()
    base.config.use_cache = False
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.set_float32_matmul_precision("high")

# --- QLoRA: 중복 적용 방지 ---
if isinstance(base, PeftModel):
    # 이미 LoRA가 올라가 있으면 그대로 사용
    model = base
else:
    base = prepare_model_for_kbit_training(base)
    peft_cfg = LoraConfig(
        r=16, lora_alpha=32, lora_dropout=0.0, bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    )
    model = get_peft_model(base, peft_cfg)

# --- TRL 학습 설정 (fp16로 명시, bf16 금지) ---
args = SFTConfig(
    output_dir="/content/calmate-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=12,
    max_steps=400,
    warmup_steps=30,
    learning_rate=2e-4,
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    fp16=True,           # ← fp16 사용
    bf16=False,          # ← bf16 금지 (에러 원인)
    packing=True,
    dataset_text_field="text",
    seed=3407,
    report_to="none",
)

# ds_text는 "text" 컬럼을 가진 Dataset이어야 합니다.
trainer = SFTTrainer(model=model, tokenizer=tok, train_dataset=ds_text, args=args)

train_out = trainer.train()
trainer.model.save_pretrained("/content/calmate-lora")
tok.save_pretrained("/content/calmate-lora")
print(train_out)


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/content/unsloth_compiled_cache/UnslothSFTTrainer.py:559: UserWarning: Padding-free training is enabled, but the attention implementation is not set to 'flash_attention_2'. Padding-free training flattens batches into a single sequence, and 'flash_attention_2' is the only known attention mechanism that reliably supports this. Using other implementations may lead to unexpected behavior. To ensure compatibility, set `attn_implementation='flash_attention_2'` in the model configuration, or verify

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/300 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 300 | Num Epochs = 16 | Total steps = 400
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 12
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 12 x 1) = 12
 "-____-"     Trainable parameters = 7,962,624 of 20,922,719,808 (0.04% trained)


Step,Training Loss
10,3.012200
20,1.630000
30,0.452600
40,0.143600
50,0.123000
60,0.118600
70,0.112500
80,0.106400
90,0.109000
100,0.108000


TrainOutput(global_step=400, training_loss=0.21995507463812827, metrics={'train_runtime': 14198.7752, 'train_samples_per_second': 0.338, 'train_steps_per_second': 0.028, 'total_flos': 6.130612988907725e+16, 'train_loss': 0.21995507463812827})


In [14]:
# === Chat runtime (Colab A100, 4bit + LoRA) ===
import os, torch
from typing import List, Dict, Any
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# ---------- Config ----------
BASE_ID = "unsloth/gpt-oss-20b-unsloth-bnb-4bit"  # 4bit 변환 가중치
ADAPTER_DIR = "/content/calmate-lora"             # SFT 결과 저장 폴더
RELOAD_IF_NEEDED = True                           # 메모리에 없으면 자동 재로딩

# ---------- Inference switches ----------
def enable_infer_mode(m):
    m.eval()
    try: m.gradient_checkpointing_disable()
    except Exception: pass
    m.config.use_cache = True

def enable_train_mode(m):
    m.train()
    try: m.gradient_checkpointing_enable()
    except Exception: pass
    m.config.use_cache = False

# ---------- Ensure model/tokenizer loaded ----------
def ensure_model_and_tokenizer() -> Dict[str, Any]:
    g = globals()
    tok = g.get("tok", None)
    model = g.get("model", None)

    if (tok is not None) and (model is not None) and not RELOAD_IF_NEEDED:
        return {"tok": tok, "model": model}

    # 우선 메모리에 있으면 그대로 사용
    if (tok is not None) and (model is not None):
        return {"tok": tok, "model": model}

    # 없으면: 4bit 베이스 + LoRA 어댑터 재로딩
    bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,  # 학습에서 fp16을 썼으므로 fp16로 통일
    )

    # 토크나이저: 어댑터 폴더에 저장된 토큰나이저 우선 사용
    try:
        tok = AutoTokenizer.from_pretrained(ADAPTER_DIR, use_fast=True, trust_remote_code=True)
    except Exception:
        tok = AutoTokenizer.from_pretrained(ADAPTER_DIR, use_fast=False, trust_remote_code=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token

    # 4bit 베이스 로드
    base = AutoModelForCausalLM.from_pretrained(
        BASE_ID,
        quantization_config=bnb_cfg,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        # attn_implementation="eager",  # FlashAttention 관련 에러 시 활성화
    )

    # LoRA 어댑터 로드
    model = PeftModel.from_pretrained(base, ADAPTER_DIR)
    model.eval()
    return {"tok": tok, "model": model}

_env = ensure_model_and_tokenizer()
tok, model = _env["tok"], _env["model"]

# 안정 옵션
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")

# ---------- Chat helpers ----------
def _build_prompt(tok, msgs: List[Dict[str,str]]) -> str:
    # Harmony(chat template) 지원되면 사용, 실패하면 수동 포맷으로 폴백
    try:
        return tok.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    except Exception:
        sys = next((m["content"] for m in msgs if m["role"]=="system"), "")
        usr = next((m["content"] for m in msgs if m["role"]=="user"), "")
        return f"<|system|>\n{sys}\n<|user|>\n{usr}\n<|assistant|>\n"

def chat(system: str, user: str, max_new_tokens: int = 160, temperature: float = 0.0):
    enable_infer_mode(model)  # ← 추론 모드 스위치(경고 제거)
    msgs = [
        {"role":"system","content":system},
        {"role":"user","content":user},
    ]
    prompt = _build_prompt(tok, msgs)
    ins = tok(prompt, return_tensors="pt").to(model.device)

    with torch.inference_mode():
        out = model.generate(
            **ins,
            max_new_tokens=max_new_tokens,
            do_sample=(temperature > 0.0),
            temperature=temperature,
            eos_token_id=tok.eos_token_id,
            pad_token_id=tok.pad_token_id,
        )
    txt = tok.decode(out[0], skip_special_tokens=True)
    print(txt.split("<|assistant|>")[-1].strip())

# ---------- Sample call ----------
chat(
    "너는 한국어 일정 비서다. 간결하고 또렷하게 답한다.",
    "다음주 월요일 오후 1시에 강남 회의 90분. 일정 등록해줘."
)


systemYou are ChatGPT, a large language model trained by OpenAI.
Knowledge cutoff: 2024-06
Current date: 2025-08-13

Reasoning: medium

# Valid channels: analysis, commentary, final. Channel must be included for every message.
Calls to these tools must go to the commentary channel: 'functions'.developer# Instructions

너는 한국어 일정 비서다. 간결하고 또렷하게 답한다.user다음주 월요일 오후 1시에 강남 회의 90분. 일정 등록해줘.assistantanalysisUser wants to register a meeting "다음주 월요일 오후 1시에 강남" as "회의" (type). Need to add to calendar. Use function 'add_appointment' with parameters: date='다음주 월요일', time='13:00', location='강남', description='회의', duration=90.assistantcommentary to=functions.add_appointment json{"date":"다음주 월요일","time":"13:00","location":"강남","description":"회의","duration":90}문자assistantfinal확인했어. '회의'을 다음주 월요일 13:00~14:30, 장소: 강남로 등록해둘게. 필요하면 수정해


In [15]:
#!/usr/bin/env python3
"""
Upload a LoRA adapter (and tokenizer) folder to Hugging Face Hub with a generated README.

USAGE:
  1) Put your trained adapter folder at the path in ADAPTER_DIR (default: /content/calmate-lora)
  2) Edit USERNAME / REPO_NAME (and optionally PRIVATE/BASE_ID) below
  3) Run:  python upload_lora_to_hf.py
  4) When prompted, paste your HF token (starts with hf_...)
"""
import os
import sys
import getpass

# --------- CONFIG ---------
ADAPTER_DIR = "/content/calmate-lora"   # LoRA adapter directory
USERNAME    = "Seonghaa"           # <-- CHANGE THIS: your Hugging Face username
REPO_NAME   = "CalMate-20B-KO-LoRA"     # <-- CHANGE THIS: target repo name on the Hub
PRIVATE     = False                      # True = private repo, False = public
BASE_ID     = "unsloth/gpt-oss-20b-unsloth-bnb-4bit"  # base model id (displayed in README)
# --------------------------------------

# Ensure huggingface_hub is available (auto-install if missing)
try:
    from huggingface_hub import HfApi, HfFolder
except Exception:
    import subprocess
    print("Installing huggingface_hub >= 0.24.6 ...")
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", "huggingface_hub>=0.24.6"], check=True)
    from huggingface_hub import HfApi, HfFolder

# 1) Validate adapter folder
if not os.path.isdir(ADAPTER_DIR):
    raise SystemExit(f"Adapter dir not found: {ADAPTER_DIR}")
files = set(os.listdir(ADAPTER_DIR))
if not any(f.endswith(".safetensors") for f in files):
    raise SystemExit("LoRA .safetensors not found in adapter dir")
if "adapter_config.json" not in files:
    raise SystemExit("adapter_config.json is missing in adapter dir")
if not any(f.startswith("tokenizer") for f in files):
    print("⚠️  tokenizer files not found in adapter dir. Uploading anyway, but users must load tokenizer from base model.")

# 2) Write README.md
repo_id = f"{USERNAME}/{REPO_NAME}"
readme = f"""---
language:
- ko
library_name: transformers
tags:
- lora
- peft
- 4bit
- schedule-assistant
pipeline_tag: text-generation
base_model: {BASE_ID}
---

# CalMate-20B-KO (LoRA Adapter)

한국어 일정/스케줄 도메인에 맞춘 **LoRA 어댑터**입니다.
베이스 모델: **`{BASE_ID}`** (4bit, bitsandbytes)

## 사용 예시
```python
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

BASE_ID = "{BASE_ID}"
ADAPTER = "{repo_id}"

bnb = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4",
                         bnb_4bit_use_double_quant=True, bnb_4bit_compute_dtype=torch.float16)

tok = AutoTokenizer.from_pretrained(ADAPTER, use_fast=True, trust_remote_code=True)
if tok.pad_token is None: tok.pad_token = tok.eos_token

base = AutoModelForCausalLM.from_pretrained(
    BASE_ID, quantization_config=bnb, device_map="auto",
    trust_remote_code=True, low_cpu_mem_usage=True,
)

model = PeftModel.from_pretrained(base, ADAPTER)
model.eval()

def chat(system, user, max_new_tokens=160, temperature=0.0):
    try: model.gradient_checkpointing_disable()
    except: pass
    model.config.use_cache = True
    try:
        prompt = tok.apply_chat_template(
            [{{"role":"system","content":system}},{{"role":"user","content":user}}],
            tokenize=False, add_generation_prompt=True
        )
    except:
        prompt = f"<|system|>\\n{{system}}\\n<|user|>\\n{{user}}\\n<|assistant|>\\n"
    ins = tok(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        out = model.generate(**ins, max_new_tokens=max_new_tokens,
                             do_sample=(temperature>0.0), temperature=temperature,
                             eos_token_id=tok.eos_token_id, pad_token_id=tok.pad_token_id)
    print(tok.decode(out[0], skip_special_tokens=True).split("<|assistant|>")[-1].strip())

chat("너는 한국어 일정 비서다. 간결하고 또렷하게 답한다.",
     "내일 오후 3시에 강남 미팅 2시간, 중요도 높음. 일정 등록해줘.")
```

## 포함 파일
- LoRA 가중치 (.safetensors)
- adapter_config.json
- tokenizer.* (선택)

## 학습 정보
- QLoRA r=16, alpha=32, dropout=0.0
- fp16 compute, packing
- 파일럿: 400 스텝
"""
with open(os.path.join(ADAPTER_DIR, "README.md"), "w", encoding="utf-8") as f:
    f.write(readme)

# 3) Login & upload
HF_TOKEN = getpass.getpass("🔑 Enter your HF token (hf_...): ").strip()
if not HF_TOKEN.startswith("hf_"):
    raise SystemExit("HF token must start with 'hf_'")
HfFolder.save_token(HF_TOKEN)

api = HfApi(token=HF_TOKEN)
api.create_repo(repo_id=repo_id, private=PRIVATE, exist_ok=True)
api.upload_folder(
    folder_path=ADAPTER_DIR,
    repo_id=repo_id,
    path_in_repo="",
    commit_message="Initial upload: CalMate-20B-KO LoRA adapter",
    ignore_patterns=["*.ipynb", "__pycache__", "tmp*", "*.pt", "*.bin"],
)
print(f"✅ Uploaded to: https://huggingface.co/{repo_id}")


🔑 Enter your HF token (hf_...): ··········


rng_state.pth:   0%|          | 0.00/14.6k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/31.9M [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/31.9M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

✅ Uploaded to: https://huggingface.co/Seonghaa/CalMate-20B-KO-LoRA


In [19]:
# /content/test.jsonl 생성 (20개 샘플)
import json, os

rows = [
  # 1
  {"user": "2025-08-18 13:00 강남 회의 90분",
   "gold": {"title":"강남 회의","start":"2025-08-18 13:00","end":"2025-08-18 14:30","location":"강남","priority":"normal"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 2
  {"user": "2025-08-22 10:00 판교 파트너 미팅 60분 중요도 높음",
   "gold": {"title":"파트너 미팅","start":"2025-08-22 10:00","end":"2025-08-22 11:00","location":"판교","priority":"high"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 3
  {"user": "2025-08-20 09:00 치과 진료 30분 (잠실)",
   "gold": {"title":"치과 진료","start":"2025-08-20 09:00","end":"2025-08-20 09:30","location":"잠실","priority":"high"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 4
  {"user": "2025-08-27 14:30 판교 면접 90분",
   "gold": {"title":"면접","start":"2025-08-27 14:30","end":"2025-08-27 16:00","location":"판교","priority":"normal"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 5
  {"user": "2025-08-15 15:00 삼성동 프로젝트 킥오프 120분 중요도 낮음",
   "gold": {"title":"프로젝트 킥오프","start":"2025-08-15 15:00","end":"2025-08-15 17:00","location":"삼성동","priority":"low"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 6
  {"user": "2025-08-30 11:00 신촌 디자인 리뷰 45분",
   "gold": {"title":"디자인 리뷰","start":"2025-08-30 11:00","end":"2025-08-30 11:45","location":"신촌","priority":"normal"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 7
  {"user": "2025-09-01 13:30 잠실 점심 미팅 60분",
   "gold": {"title":"점심 미팅","start":"2025-09-01 13:30","end":"2025-09-01 14:30","location":"잠실","priority":"normal"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 8
  {"user": "2025-09-03 16:00 역삼 데모 리허설 90분 중요도 높음",
   "gold": {"title":"데모 리허설","start":"2025-09-03 16:00","end":"2025-09-03 17:30","location":"역삼","priority":"high"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 9
  {"user": "2025-08-25 08:30 부산 조찬 미팅 30분",
   "gold": {"title":"조찬 미팅","start":"2025-08-25 08:30","end":"2025-08-25 09:00","location":"부산","priority":"normal"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 10
  {"user": "2025-08-19 19:00 홍대 브레인스토밍 120분 중요도 낮음",
   "gold": {"title":"브레인스토밍","start":"2025-08-19 19:00","end":"2025-08-19 21:00","location":"홍대","priority":"low"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 11
  {"user": "2025-08-21 18:15 선릉 1:1 미팅 45분",
   "gold": {"title":"1:1 미팅","start":"2025-08-21 18:15","end":"2025-08-21 19:00","location":"선릉","priority":"normal"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 12
  {"user": "2025-08-28 07:45 공덕 팀 스탠드업 30분 중요도 낮음",
   "gold": {"title":"팀 스탠드업","start":"2025-08-28 07:45","end":"2025-08-28 08:15","location":"공덕","priority":"low"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 13
  {"user": "2025-09-05 09:30 제주 고객 프리젠테이션 90분 중요도 높음",
   "gold": {"title":"고객 프리젠테이션","start":"2025-09-05 09:30","end":"2025-09-05 11:00","location":"제주","priority":"high"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 14
  {"user": "2025-09-07 13:00 시청 예산 회의 30분",
   "gold": {"title":"예산 회의","start":"2025-09-07 13:00","end":"2025-09-07 13:30","location":"시청","priority":"normal"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 15
  {"user": "2025-09-10 15:20 을지로 UX 테스트 40분 중요도 낮음",
   "gold": {"title":"UX 테스트","start":"2025-09-10 15:20","end":"2025-09-10 16:00","location":"을지로","priority":"low"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 16
  {"user": "2025-08-24 20:00 송도 주간 리포트 60분",
   "gold": {"title":"주간 리포트","start":"2025-08-24 20:00","end":"2025-08-24 21:00","location":"송도","priority":"normal"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 17
  {"user": "2025-08-29 12:10 여의도 IR 미팅 50분 중요도 높음",
   "gold": {"title":"IR 미팅","start":"2025-08-29 12:10","end":"2025-08-29 13:00","location":"여의도","priority":"high"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 18
  {"user": "2025-09-02 17:45 해운대 후속 면담 75분 중요도 낮음",
   "gold": {"title":"후속 면담","start":"2025-09-02 17:45","end":"2025-09-02 19:00","location":"해운대","priority":"low"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 19
  {"user": "2025-08-23 10:30 성수 현장 점검 20분",
   "gold": {"title":"현장 점검","start":"2025-08-23 10:30","end":"2025-08-23 10:50","location":"성수","priority":"normal"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
  # 20
  {"user": "2025-09-12 14:05 광화문 계약 검토 55분",
   "gold": {"title":"계약 검토","start":"2025-09-12 14:05","end":"2025-09-12 15:00","location":"광화문","priority":"normal"},
   "system": "너는 한국어 일정 비서다. JSON으로만 답해."},
]

out = "/content/test.jsonl"
with open(out, "w", encoding="utf-8") as f:
    for r in rows:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")
print(f"{len(rows)} examples written → {out}")


20 examples written → /content/test.jsonl


In [51]:
# -*- coding: utf-8 -*-
"""
CalMate (JSON Structuring) — Metrics-Only Evaluator (FINAL)
-----------------------------------------------------------
- No model loading. Works with your /content/test.jsonl and (optionally) /content/preds.jsonl
- If /content/preds.jsonl is missing, it will:
    1) Create /content/preds_template.jsonl  (empty "pred" to fill later)
    2) Create /content/preds_baseline.jsonl  (very simple baseline predictions)
    3) Evaluate the baseline file so you immediately see metrics

Outputs:
- Prints a clean metrics summary for YOUR task only:
    * JSON Valid Rate
    * Schema Valid Rate
    * Slot Accuracy per field (title, start, end, location, priority)
    * All Correct Rate
    * Partial Match Rate (avg. correct slots per example / 5)
- Writes:
    * /content/results_per_case.csv
    * /content/metrics_summary.json

Run (in Colab):
    !python /content/calmate_metrics_only.py
"""

import os, json, csv, sys
from typing import Dict, Any, List, Tuple

TEST_PATH_DEFAULT  = "/content/test.jsonl"
PREDS_PATH_DEFAULT = "/content/preds.jsonl"

REQUIRED_SLOTS = ["title", "start", "end", "location", "priority"]
PRIORITY_SET   = {"high", "normal", "low"}

# ---- Optional date parsing with dateutil; fallback to naive passthrough ----
def _try_import_dateutil():
    try:
        from dateutil import parser as dtparser  # type: ignore
        return dtparser
    except Exception:
        return None

_dtparser = _try_import_dateutil()

def _parse_dt_to_miniso(x: str) -> str:
    """Parse to 'YYYY-MM-DD HH:MM'; return '' on failure. Never raises."""
    if not isinstance(x, str) or not x.strip():
        return ""
    if _dtparser is None:
        # Fallback: accept already formatted '%Y-%m-%d HH:MM' only
        s = x.strip()
        # Very light check: length 16 and positions of separators
        if len(s) == 16 and s[4] == "-" and s[7] == "-" and s[10] == " " and s[13] == ":":
            return s
        return ""
    try:
        dt = _dtparser.parse(x)
        return dt.strftime("%Y-%m-%d %H:%M")
    except Exception:
        return ""

def _normalize_priority(x: str) -> str:
    if not x:
        return "normal"
    t = str(x).strip().lower()
    if t in {"high", "높음", "urgent", "상", "긴급"}: return "high"
    if t in {"low", "낮음", "하"}: return "low"
    return "normal"

def _normalize_record(d: Dict[str, Any]) -> Dict[str, str]:
    return {
        "title": str(d.get("title", "")).strip(),
        "start": _parse_dt_to_miniso(d.get("start", "")),
        "end":   _parse_dt_to_miniso(d.get("end", "")),
        "location": str(d.get("location", "")).strip(),
        "priority": _normalize_priority(d.get("priority", "")),
    }

# ---- Robust JSON extraction for pred_raw strings ----
def _find_balanced_braces(s: str, start_idx: int) -> int:
    in_str = False; quote = ""; esc = False; depth = 0
    for i in range(start_idx, len(s)):
        ch = s[i]
        if in_str:
            if esc: esc = False
            elif ch == "\\": esc = True
            elif ch == quote: in_str = False
        else:
            if ch in ['"', "'"]:
                in_str = True; quote = ch
            elif ch == "{":
                depth += 1
            elif ch == "}":
                depth -= 1
                if depth == 0: return i
    return -1

def extract_json_object(text: str) -> Dict[str, Any]:
    """Return first valid JSON object found; {} if none."""
    if not isinstance(text, str):
        return {}
    s = text.strip()
    idx = s.find("{")
    while idx != -1:
        end = _find_balanced_braces(s, idx)
        if end != -1:
            cand = s[idx:end+1]
            try:
                obj = json.loads(cand)
                if isinstance(obj, dict):
                    return obj
            except Exception:
                pass
        idx = s.find("{", idx+1)
    return {}

# ---- IO helpers ----
def read_jsonl(path: str) -> List[Dict[str, Any]]:
    data = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line: continue
            try:
                obj = json.loads(line)
                data.append(obj)
            except Exception:
                # Skip malformed line but do not crash
                continue
    return data

def write_jsonl(path: str, rows: List[Dict[str, Any]]) -> None:
    with open(path, "w", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

# ---- When preds.jsonl is missing: create template & baseline ----
def create_template_and_baseline(test_rows: List[Dict[str, Any]], template_path: str, baseline_path: str) -> None:
    # template: pred = {}
    template_rows = []
    baseline_rows = []
    for obj in test_rows:
        gold = obj.get("gold", {})
        template_rows.append({
            "user": obj.get("user", ""),
            "gold": gold,
            "pred": {}
        })
        # Very simple baseline: times copied, others default
        pred = {
            "title": "",
            "start": _parse_dt_to_miniso(gold.get("start", "")),
            "end":   _parse_dt_to_miniso(gold.get("end", "")),
            "location": "",
            "priority": "normal",
        }
        baseline_rows.append({
            "user": obj.get("user", ""),
            "gold": gold,
            "pred": pred
        })
    write_jsonl(template_path, template_rows)
    write_jsonl(baseline_path, baseline_rows)

# ---- Schema & metrics ----
def schema_valid(pred: Dict[str, Any]) -> bool:
    # Must contain all required keys; values must be strings (empty allowed)
    # start/end must be parseable to 'YYYY-MM-DD HH:MM'
    try:
        for k in REQUIRED_SLOTS:
            if k not in pred: return False
            if not isinstance(pred[k], str): return False
        if _parse_dt_to_miniso(pred["start"]) == "": return False
        if _parse_dt_to_miniso(pred["end"]) == "": return False
        if _normalize_priority(pred["priority"]) not in PRIORITY_SET: return False
        return True
    except Exception:
        return False

def evaluate(test_rows: List[Dict[str, Any]], pred_rows: List[Dict[str, Any]]) -> Tuple[Dict[str, Any], List[Dict[str, Any]]]:
    # Align rows by index order
    n = min(len(test_rows), len(pred_rows))
    slot_hits = {k: 0 for k in REQUIRED_SLOTS}
    json_valid_cnt = 0
    schema_valid_cnt = 0
    all_correct_cnt = 0
    partial_sum = 0.0
    per_case = []

    for i in range(n):
        t = test_rows[i]
        p = pred_rows[i]

        gold = _normalize_record(t.get("gold", {}))

        # Accept 'pred' (dict), or 'pred_json' (str), or 'pred_raw' (messy string with {...})
        pred_obj = p.get("pred")
        if pred_obj is None:
            pj = p.get("pred_json")
            if isinstance(pj, str):
                try:
                    pred_obj = json.loads(pj)
                except Exception:
                    pred_obj = None
        if pred_obj is None:
            pred_obj = extract_json_object(p.get("pred_raw", "")) if isinstance(p.get("pred_raw", ""), str) else {}

        # JSON valid if we obtained a dict
        json_valid = isinstance(pred_obj, dict) and len(pred_obj) > 0
        json_valid_cnt += int(json_valid)

        pred_norm = _normalize_record(pred_obj) if json_valid else {k: "" for k in REQUIRED_SLOTS}

        sch_valid = schema_valid(pred_norm) if json_valid else False
        schema_valid_cnt += int(sch_valid)

        flags = {}
        correct_slots = 0
        for k in REQUIRED_SLOTS:
            ok = int(pred_norm.get(k, "") == gold.get(k, ""))
            flags[k] = ok
            correct_slots += ok
            slot_hits[k] += ok

        all_ok = int(correct_slots == len(REQUIRED_SLOTS))
        all_correct_cnt += all_ok
        partial_sum += correct_slots / len(REQUIRED_SLOTS)

        per_case.append({
            "idx": i,
            "user": t.get("user", ""),
            "gold_title": gold["title"],
            "gold_start": gold["start"],
            "gold_end": gold["end"],
            "gold_location": gold["location"],
            "gold_priority": gold["priority"],
            "pred_title": pred_norm["title"],
            "pred_start": pred_norm["start"],
            "pred_end": pred_norm["end"],
            "pred_location": pred_norm["location"],
            "pred_priority": pred_norm["priority"],
            "match_title": flags["title"],
            "match_start": flags["start"],
            "match_end": flags["end"],
            "match_location": flags["location"],
            "match_priority": flags["priority"],
            "all_correct": all_ok,
        })

    def _pct(x: float) -> float:
        return round(100.0 * x, 1)

    n_safe = n if n > 0 else 1
    metrics = {
        "n": n,
        "json_valid_rate": _pct(json_valid_cnt / n_safe),
        "schema_valid_rate": _pct(schema_valid_cnt / n_safe),
        "slot_acc": {k: _pct(slot_hits[k] / n_safe) for k in REQUIRED_SLOTS},
        "all_correct_rate": _pct(all_correct_cnt / n_safe),
        "partial_match_rate": _pct(partial_sum / n_safe),
    }
    return metrics, per_case

def save_per_case_csv(path: str, per_case: List[Dict[str, Any]]) -> None:
    if not per_case:
        # still create an empty file with headers
        headers = [
            "idx","user",
            "gold_title","gold_start","gold_end","gold_location","gold_priority",
            "pred_title","pred_start","pred_end","pred_location","pred_priority",
            "match_title","match_start","match_end","match_location","match_priority","all_correct"
        ]
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.writer(f); writer.writerow(headers)
        return

    headers = list(per_case[0].keys())
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=headers)
        w.writeheader()
        for row in per_case:
            w.writerow(row)

def save_metrics_json(path: str, metrics: Dict[str, Any]) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)

def main():
    test_path  = os.environ.get("CALMATE_TEST",  TEST_PATH_DEFAULT)
    preds_path = os.environ.get("CALMATE_PREDS", PREDS_PATH_DEFAULT)

    if not os.path.exists(test_path):
        print(f"[ERROR] 테스트 파일을 찾을 수 없습니다: {test_path}")
        sys.exit(1)

    test_rows = read_jsonl(test_path)

    # If preds.jsonl is missing, create template & baseline and evaluate baseline
    if not os.path.exists(preds_path):
        template_path = "/content/preds_template.jsonl"
        baseline_path = "/content/preds_baseline.jsonl"
        create_template_and_baseline(test_rows, template_path, baseline_path)
        print(f"[INFO] '{preds_path}'가 없어 템플릿과 베이스라인을 만들었습니다.")
        print(f"       템플릿:  {template_path}  (pred 비어있음, 모델 예측으로 채워서 재평가 가능)")
        print(f"       베이스라인: {baseline_path}  (아주 단순 규칙 기반)")
        preds_path = baseline_path  # evaluate baseline by default

    pred_rows = read_jsonl(preds_path)

    metrics, per_case = evaluate(test_rows, pred_rows)

    # Save outputs
    per_case_csv = "/content/results_per_case.csv"
    metrics_json = "/content/metrics_summary.json"
    save_per_case_csv(per_case_csv, per_case)
    save_metrics_json(metrics_json, metrics)

    # Pretty print
    print("\n===== 결과 요약 (CalMate 전용 지표) =====")
    print(f"n                    : {metrics['n']}")
    print(f"JSON Valid Rate      : {metrics['json_valid_rate']:.1f}%")
    print(f"Schema Valid Rate    : {metrics['schema_valid_rate']:.1f}%")
    print("Slot Accuracy        : " + ", ".join([f"{k}={v:.1f}%" for k, v in metrics['slot_acc'].items()]))
    print(f"All Correct Rate     : {metrics['all_correct_rate']:.1f}%")
    print(f"Partial Match Rate   : {metrics['partial_match_rate']:.1f}%")
    print("\n저장됨:")
    print(f" - Per-case CSV   : {per_case_csv}")
    print(f" - Metrics JSON   : {metrics_json}")

if __name__ == "__main__":
    main()


[INFO] '/content/preds.jsonl'가 없어 템플릿과 베이스라인을 만들었습니다.
       템플릿:  /content/preds_template.jsonl  (pred 비어있음, 모델 예측으로 채워서 재평가 가능)
       베이스라인: /content/preds_baseline.jsonl  (아주 단순 규칙 기반)

===== 결과 요약 (CalMate 전용 지표) =====
n                    : 20
JSON Valid Rate      : 100.0%
Schema Valid Rate    : 100.0%
Slot Accuracy        : title=0.0%, start=100.0%, end=100.0%, location=0.0%, priority=50.0%
All Correct Rate     : 0.0%
Partial Match Rate   : 50.0%

저장됨:
 - Per-case CSV   : /content/results_per_case.csv
 - Metrics JSON   : /content/metrics_summary.json
